In [2]:
!pip install gensim

    100% |████████████████████████████████| 24.2MB 2.1MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 31.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/b8/cb/43/c0ba52baf2b0e371ec1d5b2d4685d6d24617b1391f3eeacda5
Successfully built smart-open
You are using pip version 10.0.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [41]:
from gensim.models.word2vec import Word2Vec

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re
import requests
import pandas as pd
import string

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, roc_auc_score
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to
[n

[nltk_data]    |   Unzipping corpora/sentiwordnet.zip.
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/sentence_polarity.zip.
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/shakespeare.zip.
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/sinica_treebank.zip.
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/smultron.zip.
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Unzipping corpora/state_union.zip.
[nltk_data]    | Downloading package stopwords to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Package stopwords is already up-to-dat

True

### Part 1

In [4]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "

print(whitespace_string)



  This is a    string   that has  
 a lot of  extra 
   whitespace.   


In [5]:
print (re.sub('[\s]+', ' ', whitespace_string.strip()))

This is a string that has a lot of extra whitespace.


In [6]:
r = requests.get('https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt')
dates = r.text.replace('\r', '').split('\n')
months = [re.findall(r'[A-Z][a-z]+', date)[0] for date in dates]
days = [re.findall(r'[\d]{1,2},', date)[0][:-1] for date in dates]
years = [re.findall(r'[\d]{4}', date)[0] for date in dates]
df_dates = pd.DataFrame({'Month' : months,
                        'Day' : days,
                        'Year' : years})
df_dates.head()

,Day,Month,Year
0,8,March,2015
1,15,March,2015
2,22,March,2015
3,29,March,2015
4,5,April,2015


### Part 2

In [7]:
tweets = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv')
tweets.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [8]:
tweet_list = list(tweets['SentimentText'])

In [20]:
import string
table = str.maketrans('','', string.punctuation)
stop_words = set(stopwords.words('english'))

cleaned_listings = []

for listing in tweet_list:
  # Tokenize by word
  tokens = word_tokenize(listing)
  # Make all words lowercase
  lowercase_tokens = [w.lower() for w in tokens]
  # Strip punctuation from within words
  no_punctuation = [x.translate(table) for x in lowercase_tokens]
  # Remove words that aren't alphabetic
  alphabetic = [word for word in no_punctuation if word.isalpha()]
  # Remove stopwords
  words = [w for w in alphabetic if not w in stop_words]
  # Append to list
  cleaned_listings.append(words)

In [21]:
cleaned_listings

[['sad', 'apl', 'friend'],
 ['missed', 'new', 'moon', 'trailer'],
 ['omg', 'already'],
 ['omgaga',
  'im',
  'sooo',
  'im',
  'gunna',
  'cry',
  'dentist',
  'since',
  'suposed',
  'get',
  'crown',
  'put'],
 ['think', 'mi', 'bf', 'cheating', 'tt'],
 ['worry', 'much'],
 ['juuuuuuuuuuuuuuuuussssst', 'chillin'],
 ['sunny', 'work', 'tomorrow', 'tv', 'tonight'],
 ['handed', 'uniform', 'today', 'miss', 'already'],
 ['hmmmm', 'wonder', 'number'],
 ['must', 'think', 'positive'],
 ['thanks', 'haters', 'face', 'day'],
 ['weekend', 'sucked', 'far'],
 ['jb', 'isnt', 'showing', 'australia'],
 ['ok', 'thats', 'win'],
 ['lt', 'way', 'feel', 'right'],
 ['awhhe', 'man', 'completely', 'useless', 'rt', 'funny', 'twitter', 'http'],
 ['feeling',
  'strangely',
  'fine',
  'gon',
  'na',
  'go',
  'listen',
  'semisonic',
  'celebrate'],
 ['huge', 'roll', 'thunder', 'scary'],
 ['cut',
  'beard',
  'growing',
  'well',
  'year',
  'gon',
  'na',
  'start',
  'shaunamanu',
  'happy',
  'meantime'],
 ['sa

In [22]:
tweets['tokens'] = cleaned_listings

In [31]:
cleaned_tweets = []
for listing in cleaned_listings:
  new_listing = " ".join(listing)
  cleaned_tweets.append(new_listing)
  
print(cleaned_tweets)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
tweets['cleaned_tweets'] = cleaned_tweets

In [34]:
tweets

,Sentiment,SentimentText,tokens,cleaned_tweets
0,0,is so sad for my APL frie...,"[sad, apl, friend]",sad apl friend
1,0,I missed the New Moon trail...,"[missed, new, moon, trailer]",missed new moon trailer
2,1,omg its already 7:30 :O,"[omg, already]",omg already
3,0,.. Omgaga. Im sooo im gunna CRy. I'...,"[omgaga, im, sooo, im, gunna, cry, dentist, si...",omgaga im sooo im gunna cry dentist since supo...
4,0,i think mi bf is cheating on me!!! ...,"[think, mi, bf, cheating, tt]",think mi bf cheating tt
5,0,or i just worry too much?,"[worry, much]",worry much
6,1,Juuuuuuuuuuuuuuuuussssst Chillin!!,"[juuuuuuuuuuuuuuuuussssst, chillin]",juuuuuuuuuuuuuuuuussssst chillin
7,0,Sunny Again Work Tomorrow :-| ...,"[sunny, work, tomorrow, tv, tonight]",sunny work tomorrow tv tonight
8,1,handed in my uniform today . i miss you ...,"[handed, uniform, today, miss, already]",handed uniform today miss already
9,1,hmmmm.... i wonder how she my number @-),"[hmmmm, wonder, number]",hmmmm wonder number


In [ ]:
#How should TF-IDF scores be interpreted? How are they calculated?

#The purpose of TF-IDF is to find what is unique to each document. 
# Because of this we will penalize the term frequencies of words that
# are common across all documents which will allow for each document's
# most different topics to rise to the top.

# Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency 
# (TF), aka. the number of times a word appears in a document, divided by the total number of words in
# that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of 
# the number of the documents in the corpus divided by the number of documents where the specific term appears.

# TF: Term Frequency, which measures how frequently a term occurs in a document. Since every document is different 
# in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the 
# term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of 
# normalization: 

# TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).

# IDF: Inverse Document Frequency, which measures how important a term is. While computing TF, all terms 
# are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear 
# a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, 
# by computing the following: 

# IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


### Part 3

In [35]:
X = tweets.cleaned_tweets
y = tweets.Sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
vectorizer = CountVectorizer(max_features=5000, ngram_range=(1,2), stop_words='english')

vectorizer.fit(X_train)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [37]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(79991, 5000)


,aalaap,aaron,abby,abc,able,able make,absolute,absolutely,abt,ac,...,yuck,yum,yummy,yup,yur,zealand,zero,zombie,zone,zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(19998, 5000)


,aalaap,aaron,abby,abc,able,able make,absolute,absolutely,abt,ac,...,yuck,yum,yummy,yup,yur,zealand,zero,zombie,zone,zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

In [42]:
print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')


Train Accuracy: 0.7798502331512295
Test Accuracy: 0.7496749674967497


In [53]:
cleaned_tweets

['sad apl friend',
 'missed new moon trailer',
 'omg already',
 'omgaga im sooo im gunna cry dentist since suposed get crown put',
 'think mi bf cheating tt',
 'worry much',
 'juuuuuuuuuuuuuuuuussssst chillin',
 'sunny work tomorrow tv tonight',
 'handed uniform today miss already',
 'hmmmm wonder number',
 'must think positive',
 'thanks haters face day',
 'weekend sucked far',
 'jb isnt showing australia',
 'ok thats win',
 'lt way feel right',
 'awhhe man completely useless rt funny twitter http',
 'feeling strangely fine gon na go listen semisonic celebrate',
 'huge roll thunder scary',
 'cut beard growing well year gon na start shaunamanu happy meantime',
 'sad iran',
 'wompppp wompp',
 'one see cause one else following pretty awesome',
 'lt sad level writing massive blog tweet myspace comp shut lost lays fetal position',
 'headed hospitol pull golf tourny place think reripped something yeah',
 'boring whats wrong please tell',
 'ca nt bothered wish could spend rest life sat going

In [56]:
type(cleaned_tweets)

list

### Part 4

In [58]:
tweet_model = Word2Vec(cleaned_listings, min_count=10, size = 200)


In [59]:
tweet_model.most_similar('twitter')


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('sent', 0.8644499182701111),
 ('facebook', 0.8485428690910339),
 ('email', 0.8334674835205078),
 ('list', 0.8269989490509033),
 ('following', 0.8196133375167847),
 ('dm', 0.8080013990402222),
 ('page', 0.8026947975158691),
 ('follow', 0.7938741445541382),
 ('link', 0.7856985926628113),
 ('account', 0.783019483089447)]